In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("../data/curated/data.csv")
df.head()

,url,postcode,longitude,latitude,price,bed,bath,parking,desc,cbd_dist
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,"LIGHT, LOCATION AND IMPRESSIVE ENHANCEMENTS In...",5.9614
1,https://www.domain.com.au/35-233-canterbury-ro...,3182,144.976195,-37.859150,360.0,1,1,1.0,PLEASE NOTE THAT THIS PROPERTY IS OFFERED UN-F...,4.1882
2,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,6.9114
3,https://www.domain.com.au/2-17-railway-parade-...,3163,145.069344,-37.891722,360.0,1,1,1.0,The Vida apartments offer quality living in a ...,14.3864
4,https://www.domain.com.au/4-653-park-street-br...,3056,144.964991,-37.778276,380.0,1,1,1.0,AFFORDABLE OPPORTUNITY IN EXCLUSIVE BRUNSWICK!...,8.7863


In [3]:
school = pd.read_csv("../data/raw/school-locations2022.csv", encoding = 'cp1252')

In [4]:
sub = school[['School_Name', 'Postal_Postcode']]  # select specific columns for further process
sub = sub.rename(columns = {'Postal_Postcode': 'postcode'})
sub.head()

,School_Name,postcode
0,Alberton Primary School,3971
1,Allansford and District Primary School,3277
2,Avoca Primary School,3467
3,Avenel Primary School,3664
4,Warrandyte Primary School,3113


In [5]:
sub = sub.groupby('postcode').count()

In [6]:
sub

,School_Name
postcode,
3000,4
3003,1
3004,4
3006,2
3008,1
...,...
3987,1
3988,1
3992,1


In [7]:
sub.isnull().sum() # check nan value

School_Name    0
dtype: int64

In [8]:
# merge school with rental properties on postcode
all_df = pd.merge(df, sub, on = 'postcode', how = 'left') 


In [9]:
all_df = all_df.rename(columns = {'School_Name': 'school_counts'})

In [10]:
all_df

,url,postcode,longitude,latitude,price,bed,bath,parking,desc,cbd_dist,school_counts
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,"LIGHT, LOCATION AND IMPRESSIVE ENHANCEMENTS In...",5.9614,6.0
1,https://www.domain.com.au/35-233-canterbury-ro...,3182,144.976195,-37.859150,360.0,1,1,1.0,PLEASE NOTE THAT THIS PROPERTY IS OFFERED UN-F...,4.1882,5.0
2,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,6.9114,6.0
3,https://www.domain.com.au/2-17-railway-parade-...,3163,145.069344,-37.891722,360.0,1,1,1.0,The Vida apartments offer quality living in a ...,14.3864,5.0
4,https://www.domain.com.au/4-653-park-street-br...,3056,144.964991,-37.778276,380.0,1,1,1.0,AFFORDABLE OPPORTUNITY IN EXCLUSIVE BRUNSWICK!...,8.7863,3.0
...,...,...,...,...,...,...,...,...,...,...,...
4007,https://www.domain.com.au/20-flourish-walk-dor...,3754,145.125547,-37.618487,350.0,2,1,1.0,Welcome to this unit featuring open plan Kitch...,37.3377,10.0
4008,https://www.domain.com.au/3-21-brickwood-st-br...,3186,144.997177,-37.895316,510.0,2,1,1.0,*** YOU MUST PRE-REGISTER TO ATTEND THE SCHEDU...,8.7118,8.0
4009,https://www.domain.com.au/16-spring-street-pra...,3181,145.003893,-37.850608,550.0,2,1,NaN,Located in a sought-after pocket renowned for ...,7.0455,3.0
4010,https://www.domain.com.au/109-39-riversdale-ro...,3122,145.029314,-37.827784,600.0,2,2,1.0,Enjoy the conveniences this property has to of...,9.1834,8.0


In [11]:
# find the index of rental properties which are furnished
furnish_idx = all_df.index[all_df['desc'].str.contains('furnish|Furnish').fillna(False)] 
furnish_idx


Int64Index([   4,   14,   40,   60,   81,   85,  122,  124,  128,  137,
            ...
            3960, 3961, 3963, 3967, 3969, 3972, 3985, 3987, 3990, 3996],
           dtype='int64', length=397)

In [12]:
furnish_lst = []
for i in range(len(all_df)):
    furnish_lst.append(0)   ## 0 represents not furnished
for i in range(0, len(furnish_lst)):
    if i in furnish_idx:
        furnish_lst[i] = 1   ## 1 represents furnished


In [13]:
all_df['furnished'] = furnish_lst
all_df.head()


,url,postcode,longitude,latitude,price,bed,bath,parking,desc,cbd_dist,school_counts,furnished
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,"LIGHT, LOCATION AND IMPRESSIVE ENHANCEMENTS In...",5.9614,6.0,0
1,https://www.domain.com.au/35-233-canterbury-ro...,3182,144.976195,-37.859150,360.0,1,1,1.0,PLEASE NOTE THAT THIS PROPERTY IS OFFERED UN-F...,4.1882,5.0,0
2,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,ARRANGE AN INSPECTION TIME ONLINE - DETAILS BE...,6.9114,6.0,0
3,https://www.domain.com.au/2-17-railway-parade-...,3163,145.069344,-37.891722,360.0,1,1,1.0,The Vida apartments offer quality living in a ...,14.3864,5.0,0
4,https://www.domain.com.au/4-653-park-street-br...,3056,144.964991,-37.778276,380.0,1,1,1.0,AFFORDABLE OPPORTUNITY IN EXCLUSIVE BRUNSWICK!...,8.7863,3.0,1


In [14]:
all_df = all_df[all_df["price"] < 5000]
all_df = all_df.drop(columns = ['desc'])
all_df.head()

,url,postcode,longitude,latitude,price,bed,bath,parking,cbd_dist,school_counts,furnished
0,https://www.domain.com.au/9-21-rockley-road-so...,3141,144.999757,-37.837338,400.0,1,1,1.0,5.9614,6.0,0
1,https://www.domain.com.au/35-233-canterbury-ro...,3182,144.976195,-37.859150,360.0,1,1,1.0,4.1882,5.0,0
2,https://www.domain.com.au/2-31-howitt-street-s...,3141,145.002817,-37.846776,320.0,1,1,NaN,6.9114,6.0,0
3,https://www.domain.com.au/2-17-railway-parade-...,3163,145.069344,-37.891722,360.0,1,1,1.0,14.3864,5.0,0
4,https://www.domain.com.au/4-653-park-street-br...,3056,144.964991,-37.778276,380.0,1,1,1.0,8.7863,3.0,1


In [18]:
all_df.to_csv(r'../data/curated/processed_data.csv', index = False)
